<a href="https://colab.research.google.com/github/Soumya-RanjanBhoi/Molecular-Property-Predictor-and-Drug-Likeness-/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 52.5 MB/s eta 0:00:00


In [ ]:
!pip install deepchem==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
import pandas as pd
import numpy as np
from rdkit.Chem import Descriptors

In [ ]:
import deepchem as dc
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, DataStructs
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
import xgboost as xgb

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.5 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
esol=pd.read_csv("/content/delaney-processed.csv")
freesolv=pd.read_csv("/content/SAMPL.csv")
lipo=pd.read_csv("/content/Lipophilicity.csv")

In [ ]:
esol.columns

##solubility

In [ ]:
esol.head()

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1


In [ ]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='Raw')
train, valid, test = datasets

df = pd.DataFrame(train.to_dataframe())

In [ ]:
def get_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'TPSA': Descriptors.TPSA(mol),
        'RingCount': Descriptors.RingCount(mol),
        "NumRotatableBonds": Descriptors.NumRotatableBonds(mol),
        "Fsp3": Descriptors.FractionCSP3(mol)
    }



In [ ]:
# def get_ecfp(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         return None
#     fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
#     arr = np.zeros((1,))
#     DataStructs.ConvertToNumpyArray(fp, arr)
#     return arr

In [ ]:
df['descriptors'] = df['ids'].apply(get_descriptors)
df = df.dropna(subset=['descriptors'])
desc_df = pd.json_normalize(df['descriptors'])
X = desc_df
y = df['y']

In [ ]:
df

,X,y,w,ids,descriptors
0,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c5efc40>,0.390413,1.0,CC(C)=CCCC(C)=CC(=O),"{'MolWt': 152.237, 'LogP': 2.878000000000001, ..."
1,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c5efba0>,0.090421,1.0,CCCC=C,"{'MolWt': 70.135, 'LogP': 1.9725, 'NumHDonors'..."
2,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c5efb00>,-2.464346,1.0,CCCCCCCCCCCCCC,"{'MolWt': 198.39399999999995, 'LogP': 5.707400..."
3,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c5efa60>,0.704920,1.0,CC(C)Cl,"{'MolWt': 78.542, 'LogP': 1.6336, 'NumHDonors'..."
4,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c5ef9c0>,1.159746,1.0,CCC(C)CO,"{'MolWt': 88.14999999999999, 'LogP': 1.0248, '..."
...,...,...,...,...,...
897,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c970770>,-0.649881,1.0,CC(=O)OCC(=O)C3(O)CCC4C2CCC1=CC(=O)CCC1(C)C2C(...,"{'MolWt': 402.48700000000025, 'LogP': 2.560600..."
898,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c9706d0>,-0.388598,1.0,c3ccc2nc1ccccc1cc2c3,"{'MolWt': 179.22199999999998, 'LogP': 3.388000..."
899,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c970590>,-0.654719,1.0,Nc2cccc3nc1ccccc1cc23,"{'MolWt': 194.23700000000002, 'LogP': 2.970200..."
900,<rdkit.Chem.rdchem.Mol object at 0x7c5d8c8e04f0>,-0.311180,1.0,C1CCCCCC1,"{'MolWt': 98.18900000000001, 'LogP': 2.7307000..."


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [10, None],
    'min_samples_split': [2, 5]
}
grid = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
grid.fit(X_train_scaled, y_train)


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, None], 'min_samples_split': [2, 5],
                         'n_estimators': [100, 300]})

In [ ]:
model = grid.best_estimator_

In [ ]:
y_pred = model.predict(X_test_scaled)

print("RMSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

RMSE: 0.12930845362209395
R² Score: 0.8773845422724103


In [ ]:
X_train.head()

,MolWt,LogP,NumHDonors,NumHAcceptors,TPSA,RingCount,NumRotatableBonds,Fsp3
10,96.944,1.93520,0,0,0.00,0,0,0.00
375,278.348,3.60040,0,4,52.60,1,8,0.50
247,130.231,2.33760,1,1,20.23,0,4,1.00
388,134.222,3.11842,0,0,0.00,1,1,0.40
433,297.656,3.14430,0,6,70.83,1,5,0.25


In [ ]:
def prediction(model,smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    else:
        molwt = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        h_donor = Descriptors.NumHDonors(mol)
        h_acceptor = Descriptors.NumHAcceptors(mol)
        tspa = Descriptors.TPSA(mol)
        ring = Descriptors.RingCount(mol)
        rotate_bond=Descriptors.NumRotatableBonds(mol)
        fsp=Descriptors.FractionCSP3(mol)


        print(molwt,logp,h_donor,h_acceptor,tspa,ring,rotate_bond,fsp)

        trasn=scaler.transform([[molwt, logp, h_donor, h_acceptor, tspa, ring,rotate_bond,fsp]])

        return model.predict(trasn)

In [ ]:
prediction(model,'CC(C)=CCCC(C)=CC(=O)')

152.237 2.878000000000001 0 1 17.07 0 4 0.5


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([0.24375546])

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    max_depth = trial.suggest_int("max_depth", 10, 30)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)

    bootstrap = trial.suggest_categorical("bootstrap", [True, False])

    max_features = trial.suggest_float("max_features", 0.3, 1.0)  if bootstrap else None

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42,
        n_jobs=-1
    )
    score = cross_val_score(model, X_train_scaled, y_train, cv=3,n_jobs=-1).mean()

    return score

In [ ]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective,n_trials=80)

In [ ]:
print("Best trial parameters:", study.best_trial.params)

In [ ]:
study.best_trial

In [ ]:
model1=RandomForestRegressor(**study.best_trial.params)

In [ ]:
model1.fit(X_train_scaled,y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

print("RMSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

In [ ]:
prediction(model1,"Cc1ccsc1")

In [ ]:
df_test=test.to_dataframe()

In [ ]:
df_test['descriptors'] = df_test['ids'].apply(get_descriptors)
df_test = df_test.dropna(subset=['descriptors'])
desc_df = pd.json_normalize(df_test['descriptors'])
X_test = desc_df
y_test_ = df_test['y']

In [ ]:
X_test_scaled=scaler.transform(X_test)

In [ ]:
y_test_pred=model.predict(X_test_scaled)

In [ ]:
print("RMSE:", mean_squared_error(y_test, y_test_pred))
print("R² Score:", r2_score(y_test, y_test_pred))

RMSE: 0.12930845362209395
R² Score: 0.8773845422724103


In [ ]:
prediction(model,'Cc1occc1C(=O)Nc2ccccc2')

201.22500000000002 2.840320000000001 1 2 42.24 2 2 0.08333333333333333


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([-2.94929223])

In [ ]:
import pickle
from sklearn.compose import ColumnTransformer

In [ ]:
pickle.dump(scaler,open('sol_scaler.pkl','wb'))

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open("model_solubility.pkl", "wb"))

In [ ]:
!pip install tf_keras

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

##toxicity

In [ ]:
tox=pd.read_csv('/content/tox21.csv.gz')

In [ ]:
tox.columns

In [ ]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', split='random')
train_dataset, valid_dataset, test_dataset = datasets

df_tox=train_dataset.to_dataframe()

Streaming output truncated to the last 5000 lines.
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:03:49] DEPRECATION WARNING: please use MorganGenerator
[06:0

In [ ]:
df_tox.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       ...
       'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'ids'],
      dtype='object', length=1049)

In [ ]:
df_tox.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,w4,w5,w6,w7,w8,w9,w10,w11,w12,ids
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.053987,1.152523,0.000000,1.027081,1.192513,1.038978,1.058357,0.000000,1.064168,Clc1cccc(-c2cc(Cl)cc(Cl)c2)c1
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.053987,7.556402,19.772242,1.027081,1.192513,1.038978,1.058357,6.300945,1.064168,N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.152523,1.053270,1.027081,0.000000,1.038978,1.058357,1.188646,1.064168,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.053987,1.152523,1.053270,1.027081,0.000000,1.038978,18.135889,1.188646,1.064168,CCCCC/C=C\C/C=C\CCCCCCCC(=O)OCC
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.053987,1.152523,1.053270,1.027081,1.192513,1.038978,1.058357,1.188646,1.064168,COC(=O)c1ccc(C)cc1


In [ ]:
model = dc.models.MultitaskClassifier(
    n_tasks=len(tasks),
    n_features=train_dataset.X.shape[1],
    layer_sizes=[1000],
    weight_init_stddevs=[0.02],
    bias_init_consts=[1.0],
    dropouts=[0.25],
    learning_rate=0.001,
    model_dir='tox21_model'
)

model.fit(train_dataset, nb_epoch=100)

0.02665496349334717

In [ ]:
from deepchem.metrics import Metric
from deepchem.metrics import roc_auc_score
metric = Metric(roc_auc_score, np.mean)

In [ ]:
metric = Metric(roc_auc_score, np.mean)


val_score = model.evaluate(valid_dataset, [metric], transformers)
print("Validation ROC-AUC Score:", val_score)


test_score = model.evaluate(test_dataset, [metric], transformers)
print("Test ROC-AUC Score:", test_score)


Validation ROC-AUC Score: {'mean-roc_auc_score': np.float64(0.7570208147505785)}
Test ROC-AUC Score: {'mean-roc_auc_score': np.float64(0.7540303318712253)}


In [ ]:
from deepchem.metrics import accuracy_score
metric_arr = dc.metrics.Metric(accuracy_score, np.mean)

In [ ]:
from deepchem.metrics import accuracy_score
metric = dc.metrics.Metric(accuracy_score, np.mean, mode="classification", name="accuracy")

def objective(trial):
    layer1 = trial.suggest_int('layer1', 256, 2000)
    dropout1 = trial.suggest_float('dropout1', 0.1, 0.5)
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    stddev = trial.suggest_float('stddev', 0.01, 0.1)

    model = dc.models.MultitaskClassifier(
        n_tasks=len(tasks),
        n_features=train_dataset.X.shape[1],
        layer_sizes=[layer1],
        weight_init_stddevs=[stddev],
        bias_init_consts=[1.0],
        dropouts=[dropout1],
        learning_rate=lr,
        model_dir=None,
        seed=123
    )

    model.fit(train_dataset, nb_epoch=10)
    score = model.evaluate(valid_dataset, [metric], transformers)

    return score["accuracy"]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best hyperparameters:", study.best_params)
print("Best Accuracy:", study.best_value)


[I 2025-07-26 06:06:12,869] A new study created in memory with name: no-name-3d636de9-0400-4a0e-a420-4a1626c02099
[I 2025-07-26 06:06:15,943] Trial 0 finished with value: 0.7967817561807332 and parameters: {'layer1': 368, 'dropout1': 0.12914514939553903, 'learning_rate': 0.00037207349539294146, 'stddev': 0.019009114754872075}. Best is trial 0 with value: 0.7967817561807332.
[I 2025-07-26 06:06:30,267] Trial 1 finished with value: 0.9011082693947144 and parameters: {'layer1': 1963, 'dropout1': 0.2965505798294512, 'learning_rate': 0.005837932310347543, 'stddev': 0.07229745126703412}. Best is trial 1 with value: 0.9011082693947144.
[I 2025-07-26 06:06:41,811] Trial 2 finished with value: 0.8878942881500427 and parameters: {'layer1': 1641, 'dropout1': 0.4159843782211722, 'learning_rate': 0.007123507517004597, 'stddev': 0.05292103717257561}. Best is trial 1 with value: 0.9011082693947144.
[I 2025-07-26 06:06:52,142] Trial 3 finished with value: 0.7644927536231885 and parameters: {'layer1': 

Best hyperparameters: {'layer1': 1908, 'dropout1': 0.17979464720020577, 'learning_rate': 0.005899232115724661, 'stddev': 0.02489013120565877}
Best Accuracy: 0.9099531116794544


In [ ]:
from deepchem.metrics import Metric, roc_auc_score, accuracy_score


In [ ]:
# Best hyperparameters: {'layer1': 1908, 'dropout1': 0.17979464720020577, 'learning_rate': 0.005899232115724661, 'stddev': 0.02489013120565877}
# Best Accuracy: 0.9099531116794544

In [ ]:
roc_auc_metric = Metric(roc_auc_score, np.mean)
accuracy_metric = Metric(accuracy_score, np.mean)


In [ ]:
best_params = study.best_params

In [ ]:
best_params

{'layer1': 1908,
 'dropout1': 0.17979464720020577,
 'learning_rate': 0.005899232115724661,
 'stddev': 0.02489013120565877}

In [ ]:
final_model = dc.models.MultitaskClassifier(
    n_tasks=len(tasks),
    n_features=train_dataset.X.shape[1],
    layer_sizes=[best_params['layer1']],
    weight_init_stddevs=[best_params['stddev']],
    bias_init_consts=[1.0],
    dropouts=[best_params['dropout1']],
    learning_rate=best_params['learning_rate'],
    model_dir='final_tox_model',
    seed=123,
    use_pytorch=True
)

final_model.fit(train_dataset, nb_epoch=100)

0.19472213745117187

In [ ]:
val_roc_auc = final_model.evaluate(valid_dataset, [roc_auc_metric], transformers)
val_accuracy = final_model.evaluate(valid_dataset, [accuracy_metric], transformers)

print("Validation ROC-AUC:", val_roc_auc['mean-roc_auc_score'])
print("Validation Accuracy:", val_accuracy['mean-accuracy_score'])


test_roc_auc = final_model.evaluate(test_dataset, [roc_auc_metric], transformers)
test_accuracy = final_model.evaluate(test_dataset, [accuracy_metric], transformers)

print("Test ROC-AUC:", test_roc_auc['mean-roc_auc_score'])
print("Test Accuracy:", test_accuracy['mean-accuracy_score'])


Validation ROC-AUC: 0.7187709637702177
Validation Accuracy: 0.9049445865302642
Test ROC-AUC: 0.717989235313009
Test Accuracy: 0.8930395913154534


In [ ]:
from deepchem.feat import CircularFingerprint
from deepchem.data import NumpyDataset

In [ ]:
smiles = "C1=CC2=C3C(=C1)C=CC4=CC=CC=C4C3=CC=C2"
mol = Chem.MolFromSmiles(smiles)


featurizer = CircularFingerprint(radius=2, size=1024)
features = featurizer.featurize([mol])

n_tasks = len(tasks)
dummy_y = np.zeros((1, n_tasks))
dummy_w = np.ones((1, n_tasks))


dataset = NumpyDataset(X=np.array(features), y=dummy_y, w=dummy_w)


prediction = final_model.predict(dataset)

toxic_probs = prediction[0, :, 1]

binary_preds = (toxic_probs >= 0.7).astype(int)

print(f"\nToxicity Prediction for SMILES: {smiles}")
for task, prob, pred in zip(tasks, toxic_probs, binary_preds):
    label = "Toxic" if pred == 1 else "Non-Toxic"
    print(f"{task:20s}  Prob: {prob:.4f}  =>  {label}")



Toxicity Prediction for SMILES: C1=CC2=C3C(=C1)C=CC4=CC=CC=C4C3=CC=C2
NR-AR                 Prob: 0.0000  =>  Non-Toxic
NR-AR-LBD             Prob: 0.0000  =>  Non-Toxic
NR-AhR                Prob: 0.7111  =>  Toxic
NR-Aromatase          Prob: 0.0000  =>  Non-Toxic
NR-ER                 Prob: 1.0000  =>  Toxic
NR-ER-LBD             Prob: 1.0000  =>  Toxic
NR-PPAR-gamma         Prob: 0.0000  =>  Non-Toxic
SR-ARE                Prob: 0.0000  =>  Non-Toxic
SR-ATAD5              Prob: 0.0000  =>  Non-Toxic
SR-HSE                Prob: 0.0000  =>  Non-Toxic
SR-MMP                Prob: 1.0000  =>  Toxic
SR-p53                Prob: 0.0000  =>  Non-Toxic


[07:28:09] DEPRECATION WARNING: please use MorganGenerator


In [ ]:
smiles_list = [
    "C1=CC2=C3C(=C1)C=CC4=CC=CC=C4C3=CC=C2",
    "CCO",
    "CC(=O)OC1=CC=CC=C1C(=O)O",
    "C(C(=O)O)N"
]


featurizer = CircularFingerprint(radius=2, size=1024)
n_tasks = len(tasks)

results = []

for smi in smiles_list:
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print(f"Invalid SMILES: {smi}")
        continue
    features = featurizer.featurize([mol])
    dummy_y = np.zeros((1, n_tasks))
    dummy_w = np.ones((1, n_tasks))
    dataset = NumpyDataset(X=np.array(features), y=dummy_y, w=dummy_w)
    prediction = final_model.predict(dataset)
    toxic_probs = prediction[0, :, 1]
    binary_preds = (toxic_probs >= 0.5).astype(int)

    for task, prob, pred in zip(tasks, toxic_probs, binary_preds):
        results.append({
            "SMILES": smi,
            "Task": task,
            "Probability": prob,
            "Prediction": "Toxic" if pred else "Non-Toxic"
        })

df_results = pd.DataFrame(results)
print(df_results)


In [ ]:
def predicting_toxicity(smiles,threshold=0.5):

    featurizer = CircularFingerprint(radius=2, size=1024)
    n_tasks = len(tasks)

    results=[]


    mol=Chem.MolFromSmiles(smiles)
    if mol is None:
        return 'Invalid Smiles'

    features = featurizer.featurize([mol])
    dummy_y = np.zeros((1, n_tasks))
    dummy_w = np.ones((1, n_tasks))
    dataset = NumpyDataset(X=np.array(features), y=dummy_y, w=dummy_w)
    prediction = final_model.predict(dataset)
    toxic_probs = prediction[0, :, 1]
    binary_preds = (toxic_probs >= 0.5).astype(int)

    for task, prob, pred in zip(tasks, toxic_probs, binary_preds):
        results.append({
            "SMILES": smiles,
            "Task": task,
            "Probability": prob,
            "Prediction": "Toxic" if pred else "Non-Toxic"
        })

    df_results = pd.DataFrame(results)
    return df_results


In [ ]:
datas=predicting_toxicity("C1=CC2=C3C(=C1)C=CC4=CC=CC=C4C3=CC=C2")

In [ ]:
datas

In [ ]:
smiles=datas['SMILES'][0]

In [ ]:
toxic_prob=datas['Probability'].values

In [ ]:
task=datas['Task']

In [ ]:
def generate_toxicity_report(smiles, toxic_probs, tasks, threshold=0.5):
    assay_info = {
        "NR-AR": "Possible interference with androgen receptors (hormone disruption).",
        "NR-AR-LBD": "Binding to androgen receptor ligand domain (affects hormone signaling).",
        "NR-AhR": "Activation of aryl hydrocarbon receptor (potential carcinogen response).",
        "NR-Aromatase": "Affects estrogen synthesis (endocrine disruption).",
        "NR-ER": "Interaction with estrogen receptor (hormone disruption).",
        "NR-ER-LBD": "Binding to estrogen receptor ligand domain.",
        "NR-PPAR-gamma": "Affects metabolism and inflammation pathways.",
        "SR-ARE": "Activates antioxidant response (oxidative stress).",
        "SR-ATAD5": "DNA damage repair pathway activation (genotoxic stress).",
        "SR-HSE": "Heat shock protein activation (cellular stress response).",
        "SR-MMP": "Mitochondrial toxicity (energy metabolism disruption).",
        "SR-p53": "p53 protein activation (DNA damage and apoptosis)."
    }

    print(f"\nToxicity Report for molecule: {smiles}\n" + "-"*50)

    any_toxic = False
    for task, prob in zip(tasks, toxic_probs):
        if prob >= threshold:
            any_toxic = True
            print(f"[{task}] Toxicity Likely (Probability: {prob:.2f})")
            print(f"  - {assay_info.get(task, 'No description available.')}\n")

    if not any_toxic:
        print("No significant toxicity predicted at the given threshold.")

In [ ]:
generate_toxicity_report(smiles,toxic_prob,task,0.7)

In [ ]:
final_model.save_checkpoint(model_dir='final_tox_model_checkpoint')

In [ ]:
len(tasks)

12

In [ ]:
tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

In [ ]:
def report(smiles, final_model, tasks,threshold=0.5):

    featurizer = CircularFingerprint(radius=2, size=1024)
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string."

    features = featurizer.featurize([mol])

    if not len(features) > 0:
        return "Could not featurize the molecule."

    features_array = np.array(features)
    n_tasks = 12
    n_classes = 2


    dummy_y = np.zeros((features_array.shape[0], n_tasks))
    dummy_y_one_hot = np.zeros((features_array.shape[0], n_tasks, n_classes))
    for i in range(n_tasks):
        dummy_y_one_hot[:, i, 0] = 1

    dummy_w = np.ones((features_array.shape[0], n_tasks))

    dataset = NumpyDataset(X=features_array, y=dummy_y_one_hot, w=dummy_w)

    try:
        prediction = final_model.predict(dataset)
    except ValueError as e:
        return f"Error during prediction: {e}"


    if prediction.ndim == 3 and prediction.shape[-1] == 2:
        toxic_probs = prediction[0, :, 1]
        preds = np.argmax(prediction[0], axis=1)
    else:
        toxic_probs = np.squeeze(prediction)
        preds = (toxic_probs >= threshold).astype(int)


    assay_info = {
        "NR-AR": "Possible interference with androgen receptors (hormone disruption).",
        "NR-AR-LBD": "Binding to androgen receptor ligand domain (affects hormone signaling).",
        "NR-AhR": "Activation of aryl hydrocarbon receptor (potential carcinogen response).",
        "NR-Aromatase": "Affects estrogen synthesis (endocrine disruption).",
        "NR-ER": "Interaction with estrogen receptor (hormone disruption).",
        "NR-ER-LBD": "Binding to estrogen receptor ligand domain.",
        "NR-PPAR-gamma": "Affects metabolism and inflammation pathways.",
        "SR-ARE": "Activates antioxidant response (oxidative stress).",
        "SR-ATAD5": "DNA damage repair pathway activation (genotoxic stress).",
        "SR-HSE": "Heat shock protein activation (cellular stress response).",
        "SR-MMP": "Mitochondrial toxicity (energy metabolism disruption).",
        "SR-p53": "p53 protein activation (DNA damage and apoptosis)."
    }

    report_lines = [f"Toxicity Report for molecule: {smiles}\n" + "-" * 10]
    any_toxic = False

    for i, task in enumerate(tasks):
        if i < len(toxic_probs):
            prob = toxic_probs[i]
            pred = preds[i]
            if pred == 1:
                any_toxic = True
                report_lines.append(f"[{task}] Toxicity Likely (Probability: {prob:.2f})")
                report_lines.append(f"  - {assay_info.get(task, 'No description available.')}\n")
        else:
            report_lines.append(f"Warning: Could not retrieve probability for task {task}")

    if not any_toxic:
        report_lines.append("No significant toxicity predicted.")

    return "\n".join(report_lines)

In [ ]:
print(report("CCNC1=NC(=NC(=N1)Cl)NCC",final_model,task,0.5))

Toxicity Report for molecule: CCNC1=NC(=NC(=N1)Cl)NCC
----------
[3] Toxicity Likely (Probability: 1.00)
  - No description available.



[06:30:34] DEPRECATION WARNING: please use MorganGenerator


In [ ]:
import tensorflow as tf

In [ ]:
core_model = final_model.model

In [ ]:
import torch
torch.save(final_model.model.state_dict(), "tox21_pytorch_model_1.pt")

In [ ]:
best_params

{'layer1': 1908,
 'dropout1': 0.17979464720020577,
 'learning_rate': 0.005899232115724661,
 'stddev': 0.02489013120565877}

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ToxicityNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=1908, output_dim=12, dropout=0.18):
        super(ToxicityNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:

core_model = final_model.model

torch.save(core_model.state_dict(), "tox21_model_state.pt")



In [ ]:
!pip install chembl_webresource-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
from chembl_webresource_client.new_client import new_client
import pandas as pd


molecule = new_client.molecule

approved_drugs = molecule.filter(max_phase=4)

drug_list = []
for drug in approved_drugs:
    if 'molecule_structures' in drug and drug['molecule_structures']:
        smiles = drug['molecule_structures']['canonical_smiles']
        chembl_id = drug['molecule_chembl_id']
        drug_list.append({'chembl_id': chembl_id, 'smiles': smiles})


df = pd.DataFrame(drug_list)
df['label'] = 1
df.to_csv("chembl_approved_drugs.csv", index=False)

print(f"Downloaded {len(df)} approved drug-like molecules.")


Downloaded 3594 approved drug-like molecules.


In [ ]:
df1=pd.read_csv('/content/drug_likness_label-1.csv')

In [ ]:
df1.head()

,Unnamed: 0,chembl_id,smiles,label
0,0,CHEMBL2,COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,1
1,1,CHEMBL3,CN1CCC[C@H]1c1cccnc1,1
2,2,CHEMBL4,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23,1
3,3,CHEMBL5,CCn1cc(C(=O)O)c(=O)c2ccc(C)nc21,1
4,4,CHEMBL6,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,1


In [ ]:
df1=pd.read_csv('/content/zinc_5k_decoy_dataset.csv')

In [ ]:

n_tasks = len(task)
n_features = train.X.shape[1]

drug_likeness_model = dc.models.MultitaskClassifier(
    n_tasks=n_tasks,
    n_features=n_features,
    layer_sizes=[1000],
    weight_init_stddevs=[0.02],
    bias_init_consts=[1.0],
    dropouts=[0.25],
    learning_rate=0.001,
    model_dir='drug_likeness_model'
)

print("Starting model training...")
drug_likeness_model.fit(train, nb_epoch=100)
print("Model training finished.")

Starting model training...
Model training finished.


In [ ]:
from deepchem.metrics import Metric, accuracy_score, roc_auc_score
import numpy as np

accuracy_metric = Metric(accuracy_score, np.mean)
roc_auc_metric = Metric(roc_auc_score, np.mean)


print("Evaluating model on validation set...")
val_scores = drug_likeness_model.evaluate(valid, [accuracy_metric, roc_auc_metric], transformer)
print("Validation Scores:", val_scores)


print("\nEvaluating model on test set...")
test_scores = drug_likeness_model.evaluate(test, [accuracy_metric, roc_auc_metric], transformer)
print("Test Scores:", test_scores)

Evaluating model on validation set...
Validation Scores: {'mean-accuracy_score': np.float64(0.9345817120622568), 'mean-roc_auc_score': np.float64(0.7924299354041743)}

Evaluating model on test set...
Test Scores: {'mean-accuracy_score': np.float64(0.943579766536965), 'mean-roc_auc_score': np.float64(0.7952854607694892)}


In [ ]:

def predict_drug_likeness_dc(smiles, model, featurizer):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES string: {smiles}")
        return None

    features = featurizer.featurize([mol])

    if features.shape[0] == 0:
        print(f"Could not featurize molecule from SMILES: {smiles}")
        return None

    n_tasks = model.n_tasks
    dummy_y = np.zeros((features.shape[0], n_tasks))
    dummy_w = np.ones((features.shape[0], n_tasks))
    dataset = NumpyDataset(X=features, y=dummy_y, w=dummy_w)

    prediction = model.predict(dataset)

    if prediction.shape[-1] == 2:
      drug_likeness_prob = prediction[0, 0, 1]
    else:

      drug_likeness_prob = prediction[0, 0]


    return drug_likeness_prob


In [ ]:

featurizer_used = CircularFingerprint(radius=2, size=1024)

smiles_to_predict = "CN1CCC[C@H]1c1cccnc1"

predicted_prob = predict_drug_likeness_dc(smiles_to_predict, drug_likeness_model, featurizer_used)

if predicted_prob is not None:
    print(f"\nPredicted drug-likeness probability for {smiles_to_predict}: {predicted_prob:.4f}")
    if predicted_prob >= 0.5:
        print("The molecule is likely drug-like.")
    else:
        print("The molecule is likely not drug-like.")


Predicted drug-likeness probability for CN1CCC[C@H]1c1cccnc1: 0.0005
The molecule is likely not drug-like.


[14:36:38] DEPRECATION WARNING: please use MorganGenerator


## molecule with drug-likness

In [ ]:
df1=pd.read_csv('/content/drug_likness_label-1.csv')

In [ ]:
df1.head()

,Unnamed: 0,chembl_id,smiles,label
0,0,CHEMBL2,COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,1
1,1,CHEMBL3,CN1CCC[C@H]1c1cccnc1,1
2,2,CHEMBL4,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23,1
3,3,CHEMBL5,CCn1cc(C(=O)O)c(=O)c2ccc(C)nc21,1
4,4,CHEMBL6,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,1


In [ ]:
df2=pd.read_csv('/content/zinc_5k_decoy_dataset.csv')

In [ ]:
df2.head()

,smiles,label
0,CC(C)O,0
1,CCOC(=O)C1=CC=CC=C1Cl,0
2,CCN(CC)CC,0
3,CCOC(=O)C1=CC=CC=C1Cl,0
4,CC(=O)OC1=CC=CC=C1C(=O)O,0


In [ ]:
df1.drop(['Unnamed: 0','chembl_id'],axis=1,inplace=True)

In [ ]:
final_df=pd.concat([df1,df2],axis=0)

In [ ]:
final_df.sample(10)

,smiles,label
2432,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1[C@@...,1
3560,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1.O=S(=O)(O)c1c...,1
2359,CCOP(=O)(OCC)SCC[N+](C)(C)C,1
1683,CN(C)C=O,0
4402,CCN(CC)CC,0
2211,CCCCCCCCCCCC(=O)O,0
1943,C1=CC(=CC=C1O)C=O,0
157,CC(N)Cc1ccccc1.O=S(=O)(O)O,1
1325,CC(C)CC(=O)O,0
3442,C1CNCCN1.C1CNCCN1.C1CNCCN1.O=C(O)CC(O)(CC(=O)O...,1


In [ ]:
final_df['label'].value_counts()

,count
label,
0,5000
1,3594


In [ ]:
from rdkit.Chem import Descriptors

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    descriptors = {
        "MolWt": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "TPSA": Descriptors.TPSA(mol),
        "NumHDonors": Descriptors.NumHDonors(mol),
        "NumHAcceptors": Descriptors.NumHAcceptors(mol),
        "NumRotatableBonds": Descriptors.NumRotatableBonds(mol),
        "RingCount": Descriptors.RingCount(mol),
        "HeavyAtomCount": Descriptors.HeavyAtomCount(mol),
        "FractionCSP3": Descriptors.FractionCSP3(mol),
        "BalabanJ": Descriptors.BalabanJ(mol)
    }

    return descriptors

In [ ]:
descriptor_list = []
valid_labels = []

for idx, row in final_df.iterrows():
    desc = compute_descriptors(row["smiles"])
    if desc:
        descriptor_list.append(desc)
        valid_labels.append(row["label"])

X = pd.DataFrame(descriptor_list)
y = np.array(valid_labels)

[08:33:05] WARNING: not removing hydrogen atom without neighbors
[08:33:05] WARNING: not removing hydrogen atom without neighbors
[08:33:05] WARNING: not removing hydrogen atom without neighbors
[08:33:05] WARNING: not removing hydrogen atom without neighbors
[08:33:05] WARNING: not removing hydrogen atom without neighbors
[08:33:05] WARNING: not removing hydrogen atom without neighbors


In [ ]:
X

,MolWt,LogP,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,RingCount,HeavyAtomCount,FractionCSP3,BalabanJ
0,383.408,1.78460,106.95,1,8,4,4,28,0.315789,1.556191
1,162.236,1.84830,16.13,0,2,1,2,12,0.500000,2.194592
2,361.373,1.54400,75.01,1,6,2,4,26,0.444444,2.004089
3,232.239,1.42302,72.19,1,4,2,2,17,0.250000,2.946744
4,357.793,3.92732,68.53,1,4,4,3,25,0.157895,2.232372
...,...,...,...,...,...,...,...,...,...,...
8589,101.193,1.34810,3.24,0,1,3,0,7,1.000000,2.992303
8590,200.322,3.99190,37.30,1,1,10,0,14,0.916667,2.884065
8591,73.095,-0.29560,20.31,0,1,1,0,5,0.666667,2.825749
8592,102.133,1.11710,37.30,1,1,2,0,7,0.800000,3.126007


In [ ]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

In [ ]:
!pip install LightBGM

ERROR: Could not find a version that satisfies the requirement LightBGM (from versions: none)
ERROR: No matching distribution found for LightBGM


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
X_scaled=ss.fit_transform(X)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [ ]:
model=LGBMClassifier(n_estimators=200, max_depth=5, learning_rate=0.01)
model.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 2860, number of negative: 4015
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000424 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1473
[LightGBM] [Info] Number of data points in the train set: 6875, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.416000 -> initscore=-0.339216
[LightGBM] [Info] Start training from score -0.339216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

LGBMClassifier(learning_rate=0.01, max_depth=5, n_estimators=200)

In [ ]:
y_pred=model.predict(x_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.9906922629435718

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       985
           1       1.00      0.98      0.99       734

    accuracy                           0.99      1719
   macro avg       0.99      0.99      0.99      1719
weighted avg       0.99      0.99      0.99      1719



In [ ]:
smiles='C1=CC=CN=C1'

In [ ]:
desc=compute_descriptors(smiles)

In [ ]:
desc

{'MolWt': 79.10199999999999,
 'LogP': 1.0816,
 'TPSA': 12.89,
 'NumHDonors': 0,
 'NumHAcceptors': 1,
 'NumRotatableBonds': 0,
 'RingCount': 1,
 'HeavyAtomCount': 6,
 'FractionCSP3': 0.0,
 'BalabanJ': np.float64(3.000000000000001)}

In [ ]:
molwt=desc['MolWt']
logp=desc['LogP']
tspa=desc['TPSA']
h_donor=desc['NumHDonors']
h_acc=desc['NumHAcceptors']
ring_bond=desc['NumRotatableBonds']
ring_count=desc['RingCount']
atom=desc['HeavyAtomCount']
csp=desc['FractionCSP3']
bal=desc['BalabanJ']

In [ ]:
X

,MolWt,LogP,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,RingCount,HeavyAtomCount,FractionCSP3,BalabanJ
0,383.408,1.78460,106.95,1,8,4,4,28,0.315789,1.556191
1,162.236,1.84830,16.13,0,2,1,2,12,0.500000,2.194592
2,361.373,1.54400,75.01,1,6,2,4,26,0.444444,2.004089
3,232.239,1.42302,72.19,1,4,2,2,17,0.250000,2.946744
4,357.793,3.92732,68.53,1,4,4,3,25,0.157895,2.232372
...,...,...,...,...,...,...,...,...,...,...
8589,101.193,1.34810,3.24,0,1,3,0,7,1.000000,2.992303
8590,200.322,3.99190,37.30,1,1,10,0,14,0.916667,2.884065
8591,73.095,-0.29560,20.31,0,1,1,0,5,0.666667,2.825749
8592,102.133,1.11710,37.30,1,1,2,0,7,0.800000,3.126007


In [ ]:
param=[molwt,logp,tspa,h_donor,h_acc,ring_bond,ring_count,atom,csp,bal]

In [ ]:
param=ss.transform(np.array(param).reshape(1, -1))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
prediction = model.predict_proba(param)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
param

array([[-0.59367589, -0.1772547 , -0.4493373 , -0.43035879, -0.43295138,
        -0.56934289, -0.26971551, -0.56921246, -1.4661035 ,  0.76313308]])

In [ ]:
prediction

array([[0.93509821, 0.06490179]])

In [ ]:
print(f"The predicted class is: {prediction[0]}")
if prediction[0] == 1:
    print("This molecule is predicted to be drug-like.")
else:
    print("This molecule is predicted not to be drug-like.")

The predicted class is: 1
This molecule is predicted to be drug-like.


In [ ]:
import lightgbm as lgb
from sklearn.metrics import log_loss

def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
    }

    model = LGBMClassifier(**params, n_estimators=500)
    model.fit(x_train, y_train,
              eval_set=[(x_test,y_test)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(30, verbose=False)])

    preds = model.predict_proba(x_test)
    loss = log_loss(y_test, preds)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best Trial:")
print(study.best_trial.params)

[I 2025-07-25 11:05:25,591] A new study created in memory with name: no-name-55b736e3-317c-4ded-9132-399d39782997
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-25 11:05:25,740] Trial 0 finished with value: 0.012595246689779915 and parameters: {'learning_rate': 0.19034925865835267, 'num_leaves': 227, 'max_depth': 17, 'min_child_samples': 93, 'subsample': 0.6267865222517714, 'colsample_bytree': 0.8463847896904172, 'reg_alpha': 0.26570623238448443, 'reg_lambda': 0.5856285956483163}. Best is trial 0 with value: 0.012595246689779915.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-25 11:05:26,404] Trial 1 finished with value: 0.013898331654661335 and parameters: {'learning_rate': 0.016296

Best Trial:
{'learning_rate': 0.029602699018357163, 'num_leaves': 207, 'max_depth': 15, 'min_child_samples': 91, 'subsample': 0.7019967662470535, 'colsample_bytree': 0.9050395585568829, 'reg_alpha': 0.22479182834567324, 'reg_lambda': 0.0021226122849353126}


In [ ]:
best_param=study.best_params

In [ ]:
best_param

{'learning_rate': 0.029602699018357163,
 'num_leaves': 207,
 'max_depth': 15,
 'min_child_samples': 91,
 'subsample': 0.7019967662470535,
 'colsample_bytree': 0.9050395585568829,
 'reg_alpha': 0.22479182834567324,
 'reg_lambda': 0.0021226122849353126}

In [ ]:
model = LGBMClassifier(**best_params)

In [ ]:
model.fit(x_train,y_train)

[LightGBM] [Warning] Unknown parameter: layer1
[LightGBM] [Warning] Unknown parameter: dropout1
[LightGBM] [Warning] Unknown parameter: stddev
[LightGBM] [Warning] Unknown parameter: layer1
[LightGBM] [Warning] Unknown parameter: dropout1
[LightGBM] [Warning] Unknown parameter: stddev
[LightGBM] [Info] Number of positive: 2860, number of negative: 4015
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1473
[LightGBM] [Info] Number of data points in the train set: 6875, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.416000 -> initscore=-0.339216
[LightGBM] [Info] Start training from score -0.339216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

LGBMClassifier(dropout1=0.17979464720020577, layer1=1908,
               learning_rate=0.005899232115724661, stddev=0.02489013120565877)

In [ ]:
pickle.dump(model,open('drug_likness.pkl','wb'))

In [ ]:
pickle.dump(ss,open('drug_scaler.pkl','wb'))

In [ ]:
def feature_extract(smiles):

    mol=Chem.MolFromSmiles(smiles)
    if mol is None:
        return 'Invalid Smiles'

    else:
        descriptors = {
        "MolWt": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "TPSA": Descriptors.TPSA(mol),
        "NumHDonors": Descriptors.NumHDonors(mol),
        "NumHAcceptors": Descriptors.NumHAcceptors(mol),
        "NumRotatableBonds": Descriptors.NumRotatableBonds(mol),
        "RingCount": Descriptors.RingCount(mol),
        "HeavyAtomCount": Descriptors.HeavyAtomCount(mol),
        "FractionCSP3": Descriptors.FractionCSP3(mol),
        "BalabanJ": Descriptors.BalabanJ(mol)
    }

    param=[descriptors['MolWt'],descriptors['LogP'],descriptors['TPSA'],descriptors['NumHDonors'],descriptors['NumHAcceptors'],descriptors['NumRotatableBonds'],
           descriptors['RingCount'],descriptors['HeavyAtomCount'],descriptors['FractionCSP3'],descriptors['BalabanJ']]

    scaled=ss.transform([param])

    return model.predict(scaled)


In [ ]:
feature_extract("CN(C)C=O")

[LightGBM] [Warning] Unknown parameter: layer1
[LightGBM] [Warning] Unknown parameter: dropout1
[LightGBM] [Warning] Unknown parameter: stddev


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


array([0])

In [ ]:
molwt=desc['MolWt']
logp=desc['LogP']
tspa=desc['TPSA']
h_donor=desc['NumHDonors']
h_acc=desc['NumHAcceptors']
ring_bond=desc['NumRotatableBonds']
ring_count=desc['RingCount']
atom=desc['HeavyAtomCount']
csp=desc['FractionCSP3']
bal=desc['BalabanJ']